In [1]:
import pandas as pd
from scipy.io import loadmat
import altair as alt
import numpy as np
import math
from scipy.io import loadmat
from src.spinorama.load import graph_melt
from src.spinorama.graph import graph_freq, graph_params_default
from src.spinorama.analysis import compute_cea2034

In [11]:
def parse_graph_freq_princeton_mat(mat, suffix):
    """ Suffix can be either H or V """
    ir_name = 'IR_{:1s}'.format(suffix)
    fs_name = 'fs_{:1s}'.format(suffix)
    # compute Freq                                                                                                                   
    timestep = 1./mat[fs_name]
    print(timestep)
    # hummm                                                                                                                          
    freq = np.fft.fftfreq(2**14, d=timestep)
    # reduce spectrum to 0 to 24kHz                                                                                                  
    # lg = 2**14                                                                                                                     
    # 24k is lgs = int(lg/4)                                                                                                         
    # 20k is at 3414                                                                                                                 
    lgs = 3414
    xs = freq[0][0:lgs]
    #    
    df = pd.DataFrame({'Freq': xs})
    # loop over measurements (skipping the 5 increments)
    for i in range(0, 72, 1):
        # extract ir                                                                                                                 
        ir = mat[ir_name][i]
        # compute FFT                                                                                                                
        y = np.fft.fft(ir)
        ys = np.abs(y[0:lgs])
        # check for 0 (from manual: 0 means not measured)                                                                            
        if ys.max() == 0.0:
            print('skipping {0}deg'.format(i*5))
            continue
        # apply formula from paper to translate to dbFS                                                                              
        ys = 105.+np.log10(ys)*20.
        # interpolate to smooth response                                                                                             
        # s = InterpolatedUnivariateSpline(xs, ys)                                                                                   
        # pretty print label, per 5 deg increment, follow klippel labelling                                                          
        ilabel =i*5
        if ilabel > 180: 
            ilabel = ilabel-360
        label = '{:d}°'.format(ilabel)
        if ilabel == 0:
            label = 'On Axis'
        df[label] = ys
    # sort columns in increasing angle order 
    def a2v(angle):
        if angle == 'Freq':
            return -1000
        elif angle == 'On Axis':
            return 0
        else:
            return int(angle[:-1])

    df = df.reindex(columns=sorted(df.columns, key=lambda a: a2v(a)))
    # check empty case
    if 'On Axis' not in df.keys():
        return None
    # precision of measurement is ok above 500
    return df[df.Freq>=500]

In [12]:
h_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_H_IR.mat")
v_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_V_IR.mat")

In [13]:
df = parse_graph_freq_princeton_mat(v_mat, 'V')
df is None

[[1.04166667e-05]]
skipping 0deg
skipping 5deg
skipping 10deg
skipping 15deg
skipping 20deg
skipping 25deg
skipping 30deg
skipping 35deg
skipping 40deg
skipping 45deg
skipping 50deg
skipping 55deg
skipping 60deg
skipping 65deg
skipping 70deg
skipping 75deg
skipping 80deg
skipping 85deg
skipping 90deg
skipping 95deg
skipping 100deg
skipping 105deg
skipping 110deg
skipping 115deg
skipping 120deg
skipping 125deg
skipping 130deg
skipping 135deg
skipping 140deg
skipping 145deg
skipping 150deg
skipping 155deg
skipping 160deg
skipping 165deg
skipping 170deg
skipping 175deg
skipping 180deg
skipping 185deg
skipping 190deg
skipping 195deg
skipping 200deg
skipping 205deg
skipping 210deg
skipping 215deg
skipping 220deg
skipping 225deg
skipping 230deg
skipping 235deg
skipping 240deg
skipping 245deg
skipping 250deg
skipping 255deg
skipping 260deg
skipping 265deg
skipping 270deg
skipping 275deg
skipping 280deg
skipping 285deg
skipping 290deg
skipping 295deg
skipping 300deg
skipping 305deg
skipping 31

True

In [14]:
df = parse_graph_freq_princeton_mat(h_mat, 'H')
df is None

[[1.04166667e-05]]
skipping 185deg
skipping 190deg
skipping 195deg
skipping 200deg
skipping 205deg
skipping 210deg
skipping 215deg
skipping 220deg
skipping 225deg
skipping 230deg
skipping 235deg
skipping 240deg
skipping 245deg
skipping 250deg
skipping 255deg
skipping 260deg
skipping 265deg
skipping 270deg
skipping 275deg
skipping 280deg
skipping 285deg
skipping 290deg
skipping 295deg
skipping 300deg
skipping 305deg
skipping 310deg
skipping 315deg
skipping 320deg
skipping 325deg
skipping 330deg
skipping 335deg
skipping 340deg
skipping 345deg
skipping 350deg
skipping 355deg


False

In [24]:
x = df[['On Axis']]
x[x.columns[0]]

86      88.680184
87      88.585381
88      88.349471
89      88.075364
90      87.784042
          ...    
3409    62.406832
3410    62.154144
3411    61.952354
3412    61.762524
3413    61.520443
Name: On Axis, Length: 3328, dtype: float64